In [1]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: /mnt/c/users/lonni/OneDrive/Project/Final Project/Lonnie


In [2]:
files_and_dirs = os.listdir("../")
print("Files and directories:", files_and_dirs)

Files and directories: ['.git', '.gitignore', 'Giselle', 'Lonnie', 'Mike', 'Project Main', 'README.md']


In [4]:
import pandas as pd

df = pd.read_parquet('..\Project Main\data\Appliance_file_subset.parquet')

df.drop(['images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

df.head()



FileNotFoundError: [Errno 2] No such file or directory: '..\\Project Main\\data\\Appliance_file_subset.parquet'

In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Example preprocessing
def encode_review(title, description):
    review_text = title + " " + description
    return tokenizer(review_text, padding=True, truncation=True, return_tensors="tf")

# Example input data
encoded_review = encode_review("Great product", "This product exceeded my expectations.")
output = model(encoded_review['input_ids'])

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Fit the model on your training data
model.fit(train_dataset, epochs=3)


C:\Users\lonni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'keras.__internal__'

In [4]:
! pip install keras





: 

In [2]:
import numpy as np
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd


# Load your data
df = pd.read_parquet('..\Project Main\data\Appliance_file_subset.parquet')
df.drop(['images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

# Combine the title and text into one feature
df['combined_text'] = df['title'] + ' ' + df['text']

# Prepare tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# Prepare the inputs
X = df['combined_text'].tolist()
y = df['rating'].astype(int) - 1  # Ratings should be 0-based for the model (e.g., 0 for 1-star, 4 for 5-star)

# Tokenize the text inputs
X_tokenized = tokenize_function(X)

# Convert tokenized input tensors to numpy arrays
input_ids = X_tokenized['input_ids'].numpy()

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_ids, y, test_size=0.2, random_state=42)

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,
    batch_size=16
)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy}')


C:\Users\lonni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):
No module named 'keras.__internal__'

In [4]:
import numpy as np
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
import pandas as pd

In [1]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd

# GPU check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Load your data
df = pd.read_parquet('../Project Main/data/Appliance_file_subset.parquet')
df.drop(['images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)
df['combined_text'] = df['title'] + ' ' + df['text']

# Prepare tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# Prepare the inputs and labels
X = df['combined_text'].tolist()
y = tf.convert_to_tensor(df['rating'].astype(int).values - 1, dtype=tf.int32)  # Ensure labels are TensorFlow-compatible
X_tokenized = tokenize_function(X)

# Convert input_ids and attention_mask to NumPy arrays
input_ids = X_tokenized['input_ids'].numpy()
attention_mask = X_tokenized['attention_mask'].numpy()
y = y.numpy()

# Split the dataset into training and validation sets
X_train_ids, X_val_ids, X_train_mask, X_val_mask, y_train, y_val = train_test_split(
    input_ids, attention_mask, y, test_size=0.2, random_state=42
)

# Convert the data to a TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train_ids, 'attention_mask': X_train_mask}, y_train)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val_ids, 'attention_mask': X_val_mask}, y_val)).batch(16)

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model (inside GPU context)
with tf.device('/GPU:1'):
    model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=3
    )

# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation accuracy: {accuracy}')


2024-10-25 17:13:55.530313: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-25 17:13:55.539387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 17:13:55.548399: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 17:13:55.551578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-25 17:13:55.558661: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1


I0000 00:00:1729890839.649129   89969 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729890839.649888   89969 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729890839.649926   89969 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
/home/laserlon/miniconda3/envs/dev/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnin

Epoch 1/3


I0000 00:00:1729890857.231459   90143 service.cc:146] XLA service 0x7fb468812560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729890857.231503   90143 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-10-25 17:14:17.235231: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-25 17:14:17.243405: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
I0000 00:00:1729890857.284984   90143 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 281/2500 [==>...........................] - ETA: 6:29 - loss: 1.0699 - accuracy: 0.6931

KeyboardInterrupt: 

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
# Load your data
df = pd.read_parquet('../Project Main/data/Appliance_file_subset.parquet')
df.drop(['images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

print("Available devices:", tf.config.list_physical_devices())

# Combine the title and text into one feature
df['combined_text'] = df['title'] + ' ' + df['text']
 
# Prepare tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )
 
# Prepare the inputs and labels
X = df['combined_text'].tolist()
y = tf.convert_to_tensor(df['rating'].astype(int).values - 1, dtype=tf.int32)  # Ensure labels are TensorFlow-compatible
 
# Tokenize the text inputs
X_tokenized = tokenize_function(X)
 
# Convert input_ids and attention_mask to NumPy arrays
input_ids = X_tokenized['input_ids'].numpy()
attention_mask = X_tokenized['attention_mask'].numpy()
y = y.numpy()  # Convert labels to numpy as well
 
# Split the dataset into training and validation sets
X_train_ids, X_val_ids, X_train_mask, X_val_mask, y_train, y_val = train_test_split(
    input_ids, attention_mask, y, test_size=0.2, random_state=42
)
 
# Convert the data to a TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train_ids, 'attention_mask': X_train_mask}, y_train)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val_ids, 'attention_mask': X_val_mask}, y_val)).batch(16)
 
# Compile the model
# model.compile(
#     optimizer=Adam(learning_rate=5e-5),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
 
model.compile(
    optimizer='adam',  # Using a string identifier for Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
 
# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)
 
# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation accuracy: {accuracy}')

In [1]:
import tensorflow as tf
print("CUDA Version:", tf.sysconfig.get_build_info()["cuda_version"])
print("cuDNN Version:", tf.sysconfig.get_build_info()["cudnn_version"])


2024-10-25 17:46:35.873019: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-25 17:46:35.879889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 17:46:35.888225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 17:46:35.890541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-25 17:46:35.896985: I tensorflow/core/platform/cpu_feature_guar

CUDA Version: 12.3
cuDNN Version: 8


In [3]:
# Save the entire model to a file
model.save('../Project Main/data/trained_model')

# Load the model from the file
loaded_model = tf.keras.models.load_model('../Project Main/data/trained_model')


NameError: name 'model' is not defined

In [4]:
import numpy as np
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
import pandas as pd

2024-10-28 19:31:04.550318: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-28 19:31:04.559927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 19:31:04.570213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 19:31:04.573044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 19:31:04.581315: I tensorflow/core/platform/cpu_feature_guar

In [5]:
# Load data
df = pd.read_parquet('../Project Main/data/Appliance_file_subset.parquet')
df.drop(['images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

# Combine the title and text into one feature
df['combined_text'] = df['title'] + ' ' + df['text']

min_count = df['rating'].value_counts().min()
df = df.groupby('rating').sample(n=min_count, random_state=42)

df

,rating,title,text,combined_text
13311,1.0,No apperent return mecenisums to correct the w...,Was the wrong size and am still trying to figu...,No apperent return mecenisums to correct the w...
19978,1.0,Don't buy this it doesn't work.,Bought two so far and neither one worked. Plu...,Don't buy this it doesn't work. Bought two so ...
10289,1.0,THIS FILTER IS NOT COMPATIBLE WITH DA97-08006A!,THIS FILTER IS NOT COMPATIBLE WITH DA97-08006A...,THIS FILTER IS NOT COMPATIBLE WITH DA97-08006A...
42958,1.0,Don’t buy these,These pods seemed like a good idea but don’t w...,Don’t buy these These pods seemed like a good ...
49046,1.0,One Star,One star,One Star One star
...,...,...,...,...
3119,5.0,"Good product,","Good product,","Good product, Good product,"
16013,5.0,Excellence in product/setvice,Quick delivery- exactly what I needed to repla...,Excellence in product/setvice Quick delivery- ...
48566,5.0,works great!,Perfect! That’s what I like.,works great! Perfect! That’s what I like.
13996,5.0,Fit perfect,My stove looks nice again!,Fit perfect My stove looks nice again!


In [6]:

# Prepare tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors="tf"
    )
 
# Prepare the inputs and labels
X = df['combined_text'].tolist()
y = tf.convert_to_tensor(df['rating'].astype(int).values - 1, dtype=tf.int32)  # Ensure labels are TensorFlow-compatible
 
# Tokenize the text inputs
X_tokenized = tokenize_function(X)

 # Convert input_ids and attention_mask to NumPy arrays
input_ids = X_tokenized['input_ids'].numpy()
attention_mask = X_tokenized['attention_mask'].numpy()
y = y.numpy()  # Convert labels to numpy as well

/home/laserlon/miniconda3/envs/dev/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
I0000 00:00:1730158272.642794  427968 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730158272.654031  427968 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730158272.654067  427968 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA 

In [7]:

# Split the dataset into training and validation sets
X_train_ids, X_val_ids, X_train_mask, X_val_mask, y_train, y_val = train_test_split(
    input_ids, attention_mask, y, test_size=0.2, random_state=42
)
 
# Convert the data to a TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train_ids, 'attention_mask': X_train_mask}, y_train)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val_ids, 'attention_mask': X_val_mask}, y_val)).batch(16)

In [8]:
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
 
model.compile(
    optimizer='adam',  # Using a string identifier for Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
 
# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/3


2024-10-28 19:31:48.712854: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 48.00MiB (rounded to 50331648)requested by op tf_roberta_for_sequence_classification/roberta/encoder/layer_._3/attention/self/dropout_9/dropout/GreaterEqual
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-10-28 19:31:48.712903: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-10-28 19:31:48.712909: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 81, Chunks in use: 79. 20.2KiB allocated for chunks. 19.8KiB in use in bin. 585B client-requested in use in bin.
2024-10-28 19:31:48.712912: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated fo

ResourceExhaustedError: Graph execution error:

Detected at node tf_roberta_for_sequence_classification/roberta/encoder/layer_._3/attention/self/dropout_9/dropout/GreaterEqual defined at (most recent call last):
  File "/home/laserlon/miniconda3/envs/dev/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/laserlon/miniconda3/envs/dev/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/laserlon/miniconda3/envs/dev/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/laserlon/miniconda3/envs/dev/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/laserlon/miniconda3/envs/dev/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/laserlon/miniconda3/envs/dev/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/laserlon/miniconda3/envs/dev/lib/python3.10/asyncio/base_events.py", line 603, in run

In [25]:
# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation accuracy: {accuracy}')

117/117 [==============================] - 10s 81ms/step - loss: 1.6096 - accuracy: 0.1999
Validation accuracy: 0.19989310204982758


In [37]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
import tensorflow as tf

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

# Tokenize the input
input_text = "Don't buy this it doesn't work. Bought two so far and neither one worked.  Plugged in, can hear it, but no ice.  Very expensive piece of junk."
inputs = tokenizer(input_text, return_tensors='tf')

# Make the prediction
logits = model(**inputs).logits

# Process the prediction
predictions = tf.nn.softmax(logits, axis=-1)
predicted_class = tf.argmax(predictions, axis=-1).numpy()

print(f"Predicted class: {predicted_class[0]}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Predicted class: 4


In [46]:

df_rand = df.sample(frac=1).reset_index(drop=True)

loop_counter= 0
for index, row in df_rand.iterrows():
    if loop_counter >= 500:
        break
    
    # Increment the counter
    loop_counter += 1
    
    input_text = row['combined_text']
    inputs = tokenizer(input_text, return_tensors='tf')

    # Make the prediction
    logits = model(**inputs).logits

    # Process the prediction
    predictions = tf.nn.softmax(logits, axis=-1)
    predicted_class = tf.argmax(predictions, axis=-1).numpy()
    
    print(f"Predicted class: {predicted_class[0]} : Actual class: {row['rating']}")

Predicted class: 0 : Actual class: 5.0
Predicted class: 0 : Actual class: 3.0
Predicted class: 0 : Actual class: 5.0
Predicted class: 0 : Actual class: 4.0
Predicted class: 0 : Actual class: 3.0
Predicted class: 0 : Actual class: 4.0
Predicted class: 0 : Actual class: 5.0
Predicted class: 0 : Actual class: 1.0
Predicted class: 0 : Actual class: 4.0
Predicted class: 4 : Actual class: 3.0
Predicted class: 0 : Actual class: 3.0
Predicted class: 0 : Actual class: 1.0
Predicted class: 0 : Actual class: 1.0
Predicted class: 0 : Actual class: 3.0
Predicted class: 0 : Actual class: 3.0
Predicted class: 0 : Actual class: 1.0
Predicted class: 0 : Actual class: 3.0
Predicted class: 0 : Actual class: 5.0
Predicted class: 4 : Actual class: 2.0
Predicted class: 0 : Actual class: 2.0
Predicted class: 4 : Actual class: 1.0
Predicted class: 4 : Actual class: 1.0
Predicted class: 0 : Actual class: 1.0
Predicted class: 0 : Actual class: 5.0
Predicted class: 0 : Actual class: 2.0
Predicted class: 4 : Actu

In [35]:
df.iloc[1]["combined_text"]


"Don't buy this it doesn't work. Bought two so far and neither one worked.  Plugged in, can hear it, but no ice.  Very expensive piece of junk."